In [ ]:
#################################################################################
# Written by Sean Harris
# 
# Problem: 
#   It is not easy to efficiently sample training patches from gigapixel biopsy 
#   slide. A majority of the slide is non-tissue background, and most of the 
#   tissue is noncancerous.
#
# Solution: 
#   Ahead of training, generate and save a list of indices pointing to slides 
#   containing tissue (normal and tumorous). Important to not merely iterate 
#   through entire slide, as it's mostly white background. 
#
#   Instead, the entire slide is rendered in a lower magnification level (6), 
#   and then large empty regions are filtered out with edge detection. We keep 
#   regions with lots of edges, as they contain the complex organic tissue.
#
#   Finally we exhaustively iterate through the remaining regions, filtering
#   out slides with little tissue (determined by greyscale intensity), and
#   labelling slides containing tumor anotated pixels as tumorous.
#
#################################################################################

In [ ]:
#IMPORTS

!apt-get install openslide-tools
!pip install openslide-python
 
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile
import math
import random
import sklearn
from skimage.transform import resize
import cv2
from tensorflow.keras.applications.inception_v3 import preprocess_input
import random
import copy
import scipy
import torch
import torch.nn.functional as F
from skimage import feature
import xml.etree.ElementTree as ET


 
from google.colab import drive
drive.mount("/content/drive")
slide_root='/content/drive/My Drive/ADL_Final/'
 

In [ ]:
#FUNCTIONS

def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

def find_tissue_pixels(image, intensity=0.8):
    im_gray = rgb2gray(image)
    assert im_gray.shape == (image.shape[0], image.shape[1])
    indices = np.where(im_gray <= intensity)
    return list(zip(indices[0], indices[1]))


In [ ]:
slide_root='/content/drive/My Drive/ADL_Final/training/tumor/'
coord_dict = {}
input_shape = (299,299,3)
     
#Scaling factor is how many level 0 pixels there are per "pixel" in edges
#Patch level is level of magnification of patches that will be trained on

for slide_file in os.listdir(slide_root):

    slide_path = slide_root + slide_file
    slide = open_slide(slide_path)
    print(slide_file)

    patch_level = 1
    thumbnail_level = 6
    thumbnail_stride = 100
    scaling_factor = int(thumbnail_stride * slide.level_downsamples[thumbnail_level - patch_level]) 

    coord_path = '/content/drive/My Drive/ADL_Final/coords/' + str(patch_level) + '/'
    if slide_file[0:-4] + "_normal.npy" in os.listdir(coord_path):
        print('Skipped ' + slide_file)
        continue
    
    #Step 1: Detect edges in low-res whole-slide
    a = read_slide( slide, 
                    x= 0, 
                    y= 0, 
                    level=thumbnail_level, 
                    width=slide.level_dimensions[thumbnail_level][0], 
                    height=slide.level_dimensions[thumbnail_level][1])
    b = feature.canny(rgb2gray(a))
    b = scipy.ndimage.binary_closing(b)
    b = scipy.ndimage.binary_dilation(b)
    b = scipy.ndimage.binary_fill_holes(b)
    b = b.astype(int)

    #Step 2: Convolve to generate indicator matrix denoting tissue-rich regions
    conv_window = np.full((thumbnail_stride,thumbnail_stride),1)
    b = torch.tensor(np.expand_dims(b.astype(int), axis=(0,1)))
    conv_window = torch.tensor(np.expand_dims(conv_window, axis=(0,1)))
    c = F.conv2d(b, conv_window, stride=thumbnail_stride, padding=0)
    c = c.numpy().squeeze()

    #Step 3: Get tumor pixels     
    root = ET.parse('/content/drive/My Drive/ADL_Final/xml/' + slide_file[0:-4] + '.xml').getroot()
    tumor_pixels = []                                                                                
    for annotation in root[0]:
        for coordinate in annotation[0]:
            tumor_pixels.append((round(float(coordinate.get('X'))),round(float(coordinate.get('Y')))))

    #Step 4: Gather level 1 coords represented by indicator matrix
    ratio = .333
    edges = np.argwhere(c > (ratio * np.max(c)))
    edges = np.fliplr(edges).tolist()                                                   #Remember zeroth axis in numpy is actually y axis in image, so flip axes
    normal_coords = []
    tumor_coords  = []
    for edge in edges:
        x_start = edge[0] * scaling_factor
        y_start = edge[1] * scaling_factor

        x_stop = x_start + scaling_factor
        y_stop = y_start + scaling_factor
        
        for x in range(x_start,x_stop,input_shape[0]):
            for y in range(y_start,y_stop,input_shape[1]):

                patch = read_slide(slide, x = x, y = y,
                                   level  = patch_level,
                                   width  = input_shape[0],
                                   height = input_shape[1])
                tissue_pixels = find_tissue_pixels(patch)
                if len(tissue_pixels)/(input_shape[0] * input_shape[1]) < .2:
                    continue

                for pixel in tumor_pixels:
                    if x <= pixel[0] <= (x + input_shape[0]) and y <= pixel[1] <= (y + input_shape[1]):
                        tumor_coords.append((x,y))
                        break
                normal_coords.append((x,y))
    np.save(coord_path + slide_file[0:-4] + "_normal.npy" ,np.array(normal_coords))
    np.save(coord_path + slide_file[0:-4] + "_tumor.npy" ,np.array(tumor_coords))

In [ ]:
slide.level_dimensions

In [ ]:
spot = tissue_coords[random.randint(0,len(tissue_coords))]
ppp = read_slide(slide,spot[0],spot[1],level=1,width=299,height=299)
plt.imshow(ppp)

In [ ]:
slide_file[0:-4]

In [ ]:
root = ET.parse('/content/drive/My Drive/ADL_Final/xml/' + slide_file[0:-4] + '.xml').getroot()

In [ ]:

tumor_pixels = []
for annotation in root[0]:
    for coordinate in annotation[0]:
        tumor_pixels.append((round(float(coordinate.get('X'))),round(float(coordinate.get('Y')))))

tumor_pixels